In [1]:
from __future__ import print_function
import numpy as np
import os
import sys
import time
import pickle
import tensorflow as tf

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
# Configuration for TF Session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

## Guided ReLU for guided backpropogation

Source: https://github.com/adityac94/Grad_CAM_plus_plus/blob/master/misc/GuideReLU.py

In [4]:
# Replace vanila relu to guided relu to get guided backpropagation.
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops
import tensorflow as tf

@ops.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros_like(grad))

## Retrieve the dataset generator

In [5]:
index = os.path.abspath('/home/mayankm/my_projects/data/kitti/KITTI_index.csv')
test_selector = '2011_09_30'
n_test_samples = 1
batch_size = 1
buffer_size = 32
model_path = os.path.abspath('models/miscalib-model-cam03/')
model_name = 'model-22-46483'
verbosity = 1
njobs = 9

print('KITTI Index File: ' + index)
print('Model Path: ' + model_path)

KITTI Index File: /home/mayankm/my_projects/data/kitti/KITTI_index.csv
Model Path: /media/storage/my_projects/semantics_miscalib_PLR/plr_miscalib_detection/models/miscalib-model-cam03


In [6]:
# Load the dataset.
from dataset import Dataset

dataset_test = Dataset(index, selector=test_selector, internal_shuffle=True,
                       num_of_samples=n_test_samples, n_jobs=njobs, verbose=verbosity)

1230 images found in 2 folders grouped in 2 groups from index file /home/mayankm/my_projects/data/kitti/KITTI_index.csv, when applying selector '2011_09_30'.
The samplers were initialized in 30.84 sec.


In [7]:
from __future__ import unicode_literals
# Load previous scaler
scaler_path = os.path.join(model_path, 'scaler.p')
scaler = pickle.load(open(scaler_path, 'rb'), encoding='latin1')
dataset_test.set_scaler(scaler)

print('Test with %d images' % (dataset_test.n_samples))

ids_test = np.arange(dataset_test.n_samples)

Test with 1 images


/home/mayankm/.virtualenvs/plr_pytorch/lib/python3.5/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
# Create batch generators for the test sets.
from generator import Generator

gen_test = Generator(dataset_test, ids_test, batch_size=batch_size, shuffle=True,
                     buffer_size=buffer_size, verbose=verbosity)

# Define tf model.
import tensorflow as tf
tf.reset_default_graph()

# Set tensorflow to only log errors
if verbosity == 0:
    tf.logging.set_verbosity(tf.logging.ERROR)

Filling generator buffer. Done


In [9]:
# Load previous metegraph
meta_file = os.path.join(model_path, model_name + '.meta')
print("Loading model metafile: ", meta_file)
saver = tf.train.import_meta_graph(meta_file)
graph = tf.get_default_graph()

# Check input shape
print("Input shape: ", dataset_test.shape)

Loading model metafile:  /media/storage/my_projects/semantics_miscalib_PLR/plr_miscalib_detection/models/miscalib-model-cam03/model-22-46483.meta
Input shape:  (512, 1392, 3)


## Retrieve TF Tensors for Evaluation

In [10]:
# Inputs.
input_image_tf = graph.get_tensor_by_name('input_image:0')
y_true_tf = graph.get_tensor_by_name('y_true:0')

training_tf = graph.get_tensor_by_name('training:0')

loss_tf = graph.get_tensor_by_name('loss_mse:0')
error_tf = graph.get_tensor_by_name('error_mae:0')

# Global step for logging.
global_step = 0

print(input_image_tf)

Tensor("input_image:0", shape=(?, 512, 1392, 3), dtype=float32)


## Implementation of GradCAM++

In [11]:
#  get the names of all the tensors in the graph
# [n.name for n in tf.get_default_graph().as_graph_def().node]

In [12]:
target_conv_layer = graph.get_tensor_by_name('conv_block_7/conv2/Relu:0')
output_neuron = graph.get_tensor_by_name('Squeeze:0')

In [13]:
from skimage.transform import resize

def compute_gradcam_plusplus(conv_output, conv_first_grad, conv_second_grad, conv_third_grad, shape=(512, 1392)):
    """ Computes the GradCAM++
    """
    global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[0].shape[2])), axis=0)

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num/alpha_denom

    weights = np.maximum(conv_first_grad[0], 0.0)
    #normalizing the alphas
    """	
    alpha_normalization_constant = np.sum(np.sum(alphas, axis=0),axis=0)

    alphas /= alpha_normalization_constant.reshape((1,1,conv_first_grad[0].shape[2]))
    """

    alphas_thresholding = np.where(weights, alphas, 0.0)

    alpha_normalization_constant = np.sum(np.sum(alphas_thresholding, axis=0),axis=0)
    alpha_normalization_constant_processed = np.where(alpha_normalization_constant != 0.0, alpha_normalization_constant, np.ones(alpha_normalization_constant.shape))


    alphas /= alpha_normalization_constant_processed.reshape((1,1,conv_first_grad[0].shape[2]))



    deep_linearization_weights = np.sum((weights*alphas).reshape((-1,conv_first_grad[0].shape[2])),axis=0)
    
    # print deep_linearization_weights
    grad_CAM_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)

    # Passing through ReLU
    cam = np.maximum(grad_CAM_map, 0)
    cam = cam / np.max(cam) # scale 0 to 1.0   

    cam = resize(cam, shape)
    # Passing through ReLU
    cam = np.maximum(grad_CAM_map, 0)
    cam = cam / np.max(cam) # scale 0 to 1.0    
    cam = resize(cam, shape)
    
    # return the cam array
    return cam

In [14]:
import cv2
def visualize(img, cam):
  
    fig, ax = plt.subplots(nrows=2,ncols=1)

#     plt.subplot(311)
#     plt.axis("off")
#     imgplot = plt.imshow(img)
    
#     gb_viz = np.dstack((
#                 gb_viz[:, :, 2],
#                 gb_viz[:, :, 1],
#                 gb_viz[:, :, 0],
#                 ))

#     gb_viz -= np.min(gb_viz)
#     gb_viz /= gb_viz.max()
    
#     gd_img = gb_viz*np.minimum(0.25,cam).reshape(512, 1392,1)
#     x = gd_img
#     x = np.squeeze(x)
    
#     #normalize tensor
#     x -= x.mean()
#     x /= (x.std() + 1e-5)
#     x *= 0.1

#     # clip to [0, 1]
#     x += 0.5
#     x = np.clip(x, 0, 1)

#     # convert to RGB array
#     x *= 255
   
#     x = np.clip(x, 0, 255).astype('uint8')

#     plt.axis("off")
#     imgplot = plt.imshow(x, vmin = 0, vmax = 20)
    
    plt.subplot(211)
    plt.axis("off")
    cam = (cam*-1.0) + 1.0
    cam_heatmap = np.array(cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET))
    imgplot = plt.imshow(cam_heatmap)

    plt.subplot(212)
    plt.axis("off")
    cam_heatmap = cam_heatmap/255.0
    fin = (img * 0.7) + (cam_heatmap*0.3)
    imgplot = plt.imshow(img)

In [15]:
with graph.as_default():
    with graph.gradient_override_map({'Relu': 'GuidedRelu'}):
        # get the output neuron corresponding to the class of interest (label_id)
        cost = loss_tf
        
        # get last convolutional layer gradients for generating gradCAM++ visualization
        target_conv_layer_grad = tf.gradients(cost, target_conv_layer)[0]
        
        # first_derivative
        first_derivative = tf.exp(cost) * target_conv_layer_grad
        # second_derivative
        second_derivative = tf.exp(cost) * target_conv_layer_grad * target_conv_layer_grad 
        # triple_derivative
        triple_derivative = tf.exp(cost) * target_conv_layer_grad * target_conv_layer_grad * target_conv_layer_grad
        
        # get gradients with respect to input for guided backprop
        gb_grad = tf.gradients(cost, input_image_tf)

INFO:tensorflow:Restoring parameters from /media/storage/my_projects/semantics_miscalib_PLR/plr_miscalib_detection/models/miscalib-model-cam03/model-22-46483
step     1 Test: loss_mse: 0.0025 err_mae 0.0503

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [17]:
with tf.Session(config=config) as sess:
    # Initialize tf variables.
    tf.global_variables_initializer().run()

    # load the model
    model_file = os.path.join(model_path, model_name)
    saver.restore(sess, model_file)

    # Sequence of train and validation batches.
    test_step = 0
    console_output_size = 0
    
    for b in range(gen_test.n_batches):
        images_batch, labels_batch = gen_test.next()

        # Calculate validation loss.
        batch_loss, batch_error = sess.run(
            [loss_tf, error_tf],
            feed_dict={input_image_tf: images_batch,
                       y_true_tf: labels_batch})

        # Print results.
        test_step += 1
        sys.stdout.write('\b' * console_output_size)

        console_output = 'step %5d ' % test_step
        console_output += 'Test: loss_mse: %.4f err_mae %.4f' % (
            batch_loss, batch_error)

        console_output_size = len(console_output)

        sys.stdout.write(console_output)
        sys.stdout.flush()

        # Compute the GradCAM gradients
        conv_output, conv_first_grad, conv_second_grad, conv_third_grad = sess.run(
            [target_conv_layer, first_derivative, second_derivative, triple_derivative], 
            feed_dict={input_image_tf: images_batch,
                       y_true_tf: labels_batch})

        # Compute the mask image
        cam = compute_gradcam_plusplus(conv_output, conv_first_grad, conv_second_grad, conv_third_grad)

        # Compute the guided backprop 
        gb_grad_value = sess.run(
            gb_grad, 
            feed_dict={input_image_tf: images_batch,
                       y_true_tf: labels_batch})

        visualize(images_batch[0], cam)

## Model Evalutation

In [44]:
with tf.Session(config=config) as sess:
    # Initialize tf variables.
    tf.global_variables_initializer().run()

    model_file = os.path.join(model_path, model_name)
    saver.restore(sess, model_file)
    
    # Sequence of train and validation batches.
    test_loss = 0
    test_error = 0
    test_step = 0

    console_output_size = 0
    for b in range(gen_test.n_batches):
        images_batch, labels_batch = gen_test.next()

        # Calculate validation loss.
        batch_loss, batch_error = sess.run(
            [loss_tf, error_tf],
            feed_dict={input_image_tf: images_batch,
                       y_true_tf: labels_batch})
        test_loss += batch_loss
        test_error += batch_error
        test_step += 1

        # Print results.
        sys.stdout.write('\b' * console_output_size)

        console_output = 'step %5d ' % test_step
        console_output += 'Test: loss_mse: %.4f err_mae %.4f' % (
            test_loss / test_step, test_error / test_step)

        console_output_size = len(console_output)

        sys.stdout.write(console_output)
        sys.stdout.flush()

    print()

INFO:tensorflow:Restoring parameters from /media/storage/my_projects/semantics_miscalib_PLR/plr_miscalib_detection/models/miscalib-model-cam03/model-22-46483
step     1 Test: loss_mse: 0.0005 err_mae 0.0234


In [45]:
dataset_test.stop()